Load libraries

In [13]:
#imports
from __future__ import print_function
import pyart
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import numpy as np
import h5py
from time import time
import os
from netCDF4 import num2date, date2num

In [14]:
def odim_to_nc(in_fn,out_fn,kurnell_flag):

    #read fn
    radar = pyart.aux_io.read_odim_h5(in_fn, file_field_names=True) 

    #update field names
    radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    radar.fields['DBZH']['units'] = 'dBZ'
    radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    radar.fields['VRADH']['standard_name'] = 'Velocity'
    radar.fields['VRADH']['units'] = 'm/s'
    radar.fields['VRADH']['long_name'] = 'Radial Velocity of Scatterers'

    #create custom nyquist velocity field (adjusting for kurnell)
    hfile = h5py.File(in_fn, 'r')
    d1_how = hfile['dataset1']['how'].attrs
    NI = d1_how['NI']
    NI_vec = np.ones(radar.nsweeps)*NI
    if kurnell_flag == 1:
        NI_onsite = 27
        NI_vec[1:] = NI_onsite
    new_dict = {'data': NI_vec, 'units': 'meters_per_second', 'meta_group':'instrument_parameters','comments':'Unambiguous velocity','long_name':'Nyquist Velocity'}
    radar.instrument_parameters = {'nyquist_velocity':new_dict}    

    #apply dealias filter
    gatefilter = pyart.correct.GateFilter(radar)
    gatefilter.exclude_masked('DBZH')
    corr_vel = pyart.correct.dealias_region_based(radar, vel_field='VRADH', gatefilter = gatefilter, nyquist_vel=NI_vec)
    radar.add_field('VRADH_corr', corr_vel, False)

    #write to nc
    pyart.io.write_cfradial(out_fn, radar)

In [21]:
in_dirr = '/home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/'
out_dirr = '/home/meso/Desktop/KurnellTorDec2015/cfradial/Wollgng/'

fls = os.listdir(in_dirr)
fls.sort()

i = 1
for fl in fls:
    print('doing file ',i,' ',in_dirr+ fl)
    in_fn  = in_dirr+fl
    out_fn = out_dirr+fl+'.nc'
    odim_to_nc(in_fn,out_fn,0)
    i = i +1




doing file  1   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150000Wollgng.h5
doing file  2   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150006Wollgng.h5
doing file  3   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150012Wollgng.h5
doing file  4   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150018Wollgng.h5
doing file  5   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150024Wollgng.h5
doing file  6   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150030Wollgng.h5
doing file  7   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150036Wollgng.h5
doing file  8   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150042Wollgng.h5
doing file  9   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150048Wollgng.h5
doing file  10   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150054Wollgng.h5
doing file  11   /home/meso/Desktop/KurnellTorDec2015/odimh5/Wollgng/201512150100Wollgng.